# Procesamiento de Imágenes - Trabajo Práctico N°2
## Grupo 3
* Santiago Arribere
* Matías Francois
* Joaquín Gaytan
* Pablo Scheinfeld

## Ejercicio 2 - *image_aliasing_new.m*
En esta notebook se realiza el análisis del código *image_aliasing_new.m* suministrado por la cátedra, previa traducción del mismo a Python.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [5]:
%%time
# Open image
f = plt.imread('../../../assets/barbara.gif')
f = cv.cvtColor(f, cv.COLOR_BGR2GRAY)
print(f.shape)
ysize, xsize = f.shape

(580, 720)
Wall time: 14 ms


In [6]:
%%time
# Create .avi files using motion-jpeg codec (same used in MATLAB VideoWriter by default.
mov_pics = cv.VideoWriter('aliasing_pics.avi', cv.VideoWriter_fourcc('M','J','P','G'), 10, (xsize, ysize))
mov_specs = cv.VideoWriter('aliasing_specs.avi', cv.VideoWriter_fourcc('M','J','P','G'), 10, (xsize, ysize))

for xshrink in range(0,605,5): 
    
    # Shrinking image based on x axis
    desiredxsize = xsize-xshrink
    
    # Compute desired scale
    scale_shrink = desiredxsize / xsize
    
    # Perform image downsampling (shrinking)
    f2 = cv.resize(f, (int(scale_shrink * xsize), int(scale_shrink*ysize)))
    
    # Video upsampling to get original image size (fixed size)
    f3 = cv.resize(f2, (xsize, ysize))
    
    # Compute image FFT
    Fd = np.fft.fftshift(np.log(1+np.abs(np.fft.fft2(f3)))) 

    # Convert images to frames
    fr = cv.cvtColor(f3, cv.COLOR_GRAY2BGR)
    Fdr = cv.cvtColor(np.uint8(256*Fd/np.max(np.max(Fd))), cv.COLOR_GRAY2BGR)
    
    # Write frame to movie
    mov_pics.write(fr)
    mov_specs.write(Fdr)

# Close video files    
mov_pics.release()
mov_specs.release()
cv.destroyAllWindows()    

Wall time: 7.93 s


El código anterior busca mostrar la incidencia del aliasing producido al realizar el downsampling de una imagen determinada. Para observar este efecto, se emplea la imagen *barbara.gif* como ejemplo. Dentro del loop, se toma la imagen y se le realizan dos operaciones: en primer lugar, se hace un downsampling de la imagen (es decir, se achican sus dimensiones), perdiendo información. A continuación, se aplica un upsampling a esta imagen más pequeña, contemplando una escala tal que el resultado sea del mismo tamaño de la imagen original. De esta forma, se obtiene una imagen de igual tamaño, pero con menor información respecto de la imagen original. Por último, se convierte a la imagen en un frame y se agrega al archivo de video denominado *aliasing_pics.avi*. Además, se computa su FFT y se guarda en el archivo *aliasing_specs.avi*.

A medida que avanza el for loop, se emplean factores de escalamiento más pequeños, configurando un downsampling más agresivo. Esto se traduce en una pérdida de información cada vez mayor, siendo más evidente la influencia del aliasing. Este efecto se manifiesta mediante la aparición de artefactos (como por ejemplo patrones de Moire) en la imagen. Estos artefactos son más en aquellas zonas de la imagen que contienen componentes de alta frecuencia (como por ejemplo, el patrón que posee el mantel de la mesa o el entramado de la silla). Por el contrario, si se observa el brazo de la mujer no se podrán observar efectos de aliasing sobresalientes, dado que esta zona no presenta frecuencias altas en su espectro.

Respecto de la sucesión de espectros contenida en *aliasing_specs.avi* se aprecia que comienzan a aparecer repeticiones del espectro original. Esto sucede dado que al realizar el downsampling aparece una superposición de espectreos producida por la presencia de componentes por encima de la nueva frecuencia de Nyquist. Al realizar upsampling sobre esto, se maximiza el efecto de distorsión.